### Minimization of polynomial objective (with MomentTools)
Toy and Pade objective sample

In [6]:
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials, MomentTools
using MosekTools
using Random
using JuMP

Consider minimizing toy example

In [74]:
@polyvar x[1:3]

f = (x[1]+11)^2 + (x[2]+22)^4 + (x[3]+33)^2 + 44
maxdegree(f)

4

Lets use JuMP.optimizer_with_attributes function from the JuMP.jl package to group an optimizer constructor with the list of attributes <br>
https://jump.dev/JuMP.jl/stable/reference/models/#JuMP.optimizer_with_attributes

In [75]:
using JuMP
optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)

MathOptInterface.OptimizerWithAttributes(Mosek.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawParameter("QUIET") => true])

Now we can use MomentTools.minimize function <br>
http://www-sop.inria.fr/members/Bernard.Mourrain/software/MomentTools/code/2.optimization.html#MomentTools.minimize

In [77]:
obj_min, M = minimize(f, [], [], variables(f), maxdegree(f) ÷ 2, optimizer)
# obj_min, M = minimize(f, [], [], variables(f), 2, optimizer)
# obj_min, M = minimize(f, [], [], variables(f), maxdegree(f), optimizer)

(44.00000051722838, A JuMP Model
Minimization problem with:
Variables: 35
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`Vector{AffExpr}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: basis, degree, dual, index, moments, monomials, nu, type, variables, y)

In [78]:
variables(f)

3-element Vector{PolyVar{true}}:
 x₁
 x₂
 x₃

In [79]:
obj_min

44.00000051722838

In [80]:
get_minimizers(M)

3×1 Matrix{Float64}:
 -11.000004868052248
 -21.99743305188541
 -33.000006390343465

Declare functions to constract Pade objective

In [84]:
function frobenius_norm2(m)
    return tr(m * m')
end

function lindblad_rhs(rho, H, A)
    """
    Right hand side of the Lindblad master equation
    """
    return -1im * (H * rho - rho * H) + A * rho * A' - (A' * A  * rho + rho * A' * A) / 2
    
end

function pade_obj(ρ, t, H, A)
    
    obj = 0
    for i in 2:size(ρ,3)
        obj += frobenius_norm2(
            ρ[:, :, i] - ρ[:, :, 1] 
            - (t[i]-t[i-1])*lindblad_rhs((ρ[:, :, i]+ρ[:, :, i-1])/2, H, A)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

pade_obj (generic function with 1 method)

Construct sample Pade objective

In [85]:
@polyvar x[1:4]
H = [ 1.0 * x[1]              x[3] + im * x[4]
      x[3] - im * x[4]        x[2]             ]

@polyvar a[1:2, 1:2]
@polyvar b[1:2, 1:2]
A = 1.0 * a + im * b

ρ1 = [.5 -im
     +im  .5]
ρ2 = [.6 0
     0  .4]
ρ3 = [.7 0
     0  .3]
ρ4 = [.9 0
     0  .1]

ρ = [ρ1;;; ρ2;;; ρ3;;; ρ4]
t = [0.0, 0.1, 0.2, 0.3]

obj = pade_obj(ρ, t, H, A);

Use JuMP.optimizer_with_attributes and MomentTools.minimize

In [87]:
maxdegree(obj)

4

In [88]:
variables(obj)

12-element Vector{PolyVar{true}}:
 x₁
 x₂
 x₃
 x₄
 a₁₋₁
 a₂₋₁
 a₁₋₂
 a₂₋₂
 b₁₋₁
 b₂₋₁
 b₁₋₂
 b₂₋₂

In [86]:
optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
obj_min, M = minimize(obj, [], [], variables(obj), maxdegree(obj) ÷ 2, optimizer)

(1.139253199056551, A JuMP Model
Minimization problem with:
Variables: 1820
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`Vector{AffExpr}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: basis, degree, dual, index, moments, monomials, nu, type, variables, y)

In [89]:
get_minimizers(M)

12×9 Matrix{Float64}:
 -0.0125177  -0.0141248  -0.0195061  …  -0.127075    0.0229445   0.592919
 -0.0812818   0.0269903   0.0191686      0.1454     -0.0100024  -0.349966
  0.0204933   0.929546   -0.0894346      0.653755    0.483383    0.534937
 -0.17416    -0.120408    0.0734679      0.0900822  -0.112165   -0.152447
  0.225151    0.264898    0.261968      -0.638162   -0.292563    0.362057
 -0.73488    -0.698281   -0.162723   …   0.377071    0.7153      0.697751
 -1.1018     -1.03415    -0.846758       0.859905    1.06968     1.12374
 -0.031072   -0.15492    -0.0701788      0.069234    0.0841564   0.413034
 -2.09289    -1.53606    -0.541523       0.910868    1.82271     1.84462
 -0.0674014  -0.115075   -0.0946625      0.217008    0.106434    0.0139887
 -0.212415   -0.31651    -0.224247   …   0.369141    0.261193    0.03508
  0.247247    0.681834    0.202107      -0.401952   -0.453256   -0.408745

Find best candidate

In [94]:
r = real.(get_minimizers(M))
obj_min_vals = [obj(r[:,i]) for i=1:size(r)[2]]
min_x = r[:, argmin(obj_min_vals)]

12-element Vector{Float64}:
  0.1206574582022943
 -0.04056000217566395
  0.13134071685642232
 -0.029286823549031917
  0.936351124794301
  0.6763173545717279
  1.027429938345106
 -0.08358868585843786
  1.8892249548920144
 -0.32102521462601935
 -0.4767135996224034
 -0.2696050720435906

Convert polyvar objective into the normal variable objective

In [95]:
using NLopt

vars = variables(obj)

function g(a...)
    # Converting polynomial expression to function to be minimize
    obj(vars => a)
end
    
model = Model(NLopt.Optimizer)

set_optimizer_attribute(model, "algorithm", :LD_MMA)

#set_silent(model)
@variable(model, y[1:length(vars)]);

Set initial values to new variables using the best candidate minimizer:

In [97]:
for (var, init_val) in zip(y, min_x)
    set_start_value(var, init_val)
end

Perform local minimization

In [98]:
register(model, :g, length(y), g; autodiff = true)
@NLobjective(model, Min, g(y...))
JuMP.optimize!(model)
solution = vars => map(value, y)
objective_value(model)

1.139253130837866

In [99]:
min(obj_min_vals...)

3.203473415661914

Now we have the best minimizer

In [100]:
solution

PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂] => [0.07924254916569747, 0.0008549068609328754, 0.6164008440889793, -0.020447621364296214, 1.4625037536131977, 1.0885623518616947, 1.6286664463597507, -0.43159598149450695, 2.6250934409781355, -0.5954536993710784, -0.8871877800131419, -0.8388804236336037]

First construct function to perform local minimization of objective

In [103]:
using NLopt

function minimize_local(obj, min_x) # polynomial objective, and guess x candidate
    vars = variables(obj)

    function g(a...)
        # Converting polynomial expression to function to be minimize
        obj(vars => a)
    end
    
    model = Model(NLopt.Optimizer)

    set_optimizer_attribute(model, "algorithm", :LD_MMA)
    
    #set_silent(model)
    @variable(model, y[1:length(vars)]);
    
    for (var, init_val) in zip(y, min_x)
        set_start_value(var, init_val)
    end
    
    register(model, :g, length(y), g; autodiff = true)
    @NLobjective(model, Min, g(y...))
    JuMP.optimize!(model)
    solution = vars => map(value, y)
    
    return model, solution
end 

minimize_local (generic function with 1 method)

Now construct single function to perform all the process of minimization of objective

In [105]:
function minimize_global(obj)
    optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
    obj_min, M = minimize(obj, [], [], variables(obj), maxdegree(obj) ÷ 2, optimizer)
    
    r = real.(get_minimizers(M))
    obj_min_vals = [obj(r[:,i]) for i=1:size(r)[2]]
    min_x = r[:, argmin(obj_min_vals)]
    
    model, solution = minimize_local(obj, min_x) 
    
    return model, solution
    
end 

minimize_global (generic function with 1 method)

In [106]:
model, solution = minimize_global(obj)

(A JuMP Model
Minimization problem with:
Variables: 12
Objective function type: Nonlinear
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: NLopt
Names registered in the model: y, PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂] => [0.07924254916569747, 0.0008549068609328754, 0.6164008440889793, -0.020447621364296214, 1.4625037536131977, 1.0885623518616947, 1.6286664463597507, -0.43159598149450695, 2.6250934409781355, -0.5954536993710784, -0.8871877800131419, -0.8388804236336037])

In [107]:
solution

PolyVar{true}[x₁, x₂, x₃, x₄, a₁₋₁, a₂₋₁, a₁₋₂, a₂₋₂, b₁₋₁, b₂₋₁, b₁₋₂, b₂₋₂] => [0.07924254916569747, 0.0008549068609328754, 0.6164008440889793, -0.020447621364296214, 1.4625037536131977, 1.0885623518616947, 1.6286664463597507, -0.43159598149450695, 2.6250934409781355, -0.5954536993710784, -0.8871877800131419, -0.8388804236336037]